# Fitting in different orbital bases

In this tutorial, we show how one can perform orbit-fits in different coordinate bases amongst the ones supported by `orbitize`. Currently fitting in different bases is only supported in MCMC, so we will use MCMC to perform an orbit-fit in an orbital basis distinct from the default one. For a general introduction to MCMC, be sure to check out the [MCMC Introduction tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html) first!

## The "standard" and "XYZ" bases

The default way to define an orbit in `orbitize` is through what we call the 'standard basis', which consists of eight parameters: semi-major axis (sma), eccentricity (ecc), inclination (inc), argument of periastron (aop), position angle of the nodes (pan), epoch of periastron expressed as a fraction of the period past a reference epoch (tau), parallax (plx) and total system mass (mtot). Each orbital element has an associated default prior; to see how to explore and modify these priors check out the [Modifying priors tutorial](http://orbitize.info/en/latest/tutorials/Modifying_Priors.html).

An alternative way to define an orbit is through its position and velocity components in XYZ space for a given epoch; we will call this the 'XYZ basis'. The orbit is thus defined with the array ($x$, $y$, $z$, $\dot{x}$, $\dot{y}$,$\dot{z}$, plx, mtot), with position coordinates measured in AU and velocity components in $\text{km s}^{-1}$. In this basis, the sky-plane coordinates ($x,y$) are the separations of the planet relative to the primary, with the positive $x$ and $y$ directions coinciding with the positive RA and Dec directions, respectively. The $z$ direction is the line-of-sight coordinate, such that movement in the positive $z$ direction causes a redshift. The default priors are uniform all uniform.

## Setting up Sampler in the XYZ basis

The easiest way to run an orbit-fit in an alternative orbital basis in `orbitize` is through the `orbitize.driver.Driver` interface. The process is exactly like initializing a regular `Driver` object, but setting the `fitting_basis` keyword to 'XYZ':

In [1]:
import numpy as np

import orbitize
from orbitize import driver
import multiprocessing as mp

filename = "{}xyz_test_data.csv".format(orbitize.DATADIR) # a file with input in radec since rn it only works for that

# system parameters
num_secondary_bodies = 1
system_mass = 1.75 # [Msol]
plx = 51.44 # [mas]
mass_err = 0.05 # [Msol]
plx_err = 0.12 # [mas]

# MCMC parameters
num_temps = 5
num_walkers = 20
num_threads = mp.cpu_count() # or a different number if you prefer


my_driver = driver.Driver(
    filename, 'MCMC', num_secondary_bodies, system_mass, plx, mass_err=mass_err, plx_err=plx_err,
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads},
    system_kwargs={'fitting_basis': 'XYZ'}
)

s = my_driver.sampler

Converting ra/dec data points in data_table to sep/pa. Original data are stored in input_table.


### (Properly) initializing walkers in the XYZ basis

In the standard basis at this point we would be ready to use the `s.run_sampler` method to start the sampling, but with the XYZ basis we have to make sure that all our walkers are initialized in a valid region of parameter space. This is because randomly generated values of ($x$, $y$, $z$, $\dot{x}$, $\dot{y}$, $\dot{z}$) can result in unbound, invalid orbits with, for example, negative eccentricities (which is not cool). This can be easily corrected with the `s.validate_xyz_positions` method:

In [2]:
s.validate_xyz_positions()

All walker positions validated.


/home/sblunt/Projects/orbitize/orbitize/basis.py:936: RuntimeWarning: invalid value encountered in arccos
  eanom = np.arccos(cos_eanom)
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/units/quantity.py:481: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


___________________

After this is done, the sampler can be run and the results saved normally:

In [3]:
total_orbits = 600 # number of steps x number of walkers (at lowest temperature)
burn_steps = 10 # steps to burn in per walker
thin = 2 # only save every 2nd step

s.run_sampler(total_orbits, burn_steps=burn_steps, thin=thin)
s.results.save_results('my_posterior.hdf5')

Starting Burn in


/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: Run

/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: Run

10/10 steps of burn-in complete
Burn in complete. Sampling posterior now.


/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: Run

/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))


/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: Run

/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))


/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: RuntimeWarning: invalid value encountered in sqrt
  tanom = 2.*np.arctan(np.sqrt((1.0 + ecc)/(1.0 - ecc))*np.tan(0.5*eanom))
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/sblunt/Projects/orbitize/orbitize/kepler.py:112: Run

30/30 steps completed
Run complete


## Loading and converting results

You can load the results as you normally would. The orbit posteriors are saved in the `results.post` attribute, and the basis you used for the fit in the `results.fitting_basis` attribute:

In [4]:
myResults = orbitize.results.Results() # create empty Results object
myResults.load_results('my_posterior.hdf5') 
print('The used basis for the fit was ', myResults.fitting_basis)
print('The posteriors are ', myResults.post)

Converting ra/dec data points in data_table to sep/pa. Original data are stored in input_table.
The used basis for the fit was  XYZ
The posteriors are  [[-1.55958816e+01 -3.20124887e+01 -7.00594561e+00 ...  5.44442171e-01
   5.15046239e+01  1.73814359e+00]
 [-1.56183873e+01 -3.21141437e+01 -3.46391968e+00 ... -5.37390717e-01
   5.14009692e+01  1.76651544e+00]
 [-1.56046654e+01 -3.21356777e+01 -3.46373591e+00 ...  1.29026237e-02
   5.13691699e+01  1.74173728e+00]
 ...
 [-1.55994750e+01 -3.20950558e+01 -2.86610260e+00 ...  4.82750370e-01
   5.14044029e+01  1.70718930e+00]
 [-1.55734128e+01 -3.20184718e+01 -3.28018029e+00 ... -3.47152166e-02
   5.15258210e+01  1.76599736e+00]
 [-1.56003120e+01 -3.21126549e+01  2.10437959e+01 ...  5.61325347e-01
   5.13884498e+01  1.76613892e+00]]


Let's convert back to the good old standard basis:

In [13]:
xyz_posterior = myResults.post

standard_posterior = myResults.system.basis.to_standard_basis(xyz_posterior)

print('My posterior in standard basis is ', standard_posterior)

My posterior in standard basis is  [[ 7.87962074e+00  6.91826954e+00  4.19938473e+00 ...  3.83578942e-01
  -6.60220250e+00  1.52778649e+00]
 [ 1.00000147e+00  1.00006101e+00  1.00057893e+00 ...  9.94686006e-01
   1.00011978e+00  9.99999278e-01]
 [ 6.16582641e-01  8.94378628e-01  7.85487334e-01 ...  5.61601645e-01
   2.20297402e+00  8.19499235e-01]
 ...
 [-1.55994750e+01 -3.20950558e+01 -2.86610260e+00 ...  4.82750370e-01
   5.14044029e+01  1.70718930e+00]
 [-1.55734128e+01 -3.20184718e+01 -3.28018029e+00 ... -3.47152166e-02
   5.15258210e+01  1.76599736e+00]
 [-1.56003120e+01 -3.21126549e+01  2.10437959e+01 ...  5.61325347e-01
   5.13884498e+01  1.76613892e+00]]


/home/sblunt/Projects/orbitize/orbitize/basis.py:936: RuntimeWarning: invalid value encountered in arccos
  eanom = np.arccos(cos_eanom)
/data/user/sblunt/miniconda3/envs/python3.7/lib/python3.7/site-packages/astropy/units/quantity.py:481: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


And we're done! Enjoy the XYZ basis.